# News Preprocessing Using Ckiplab NLP Package

多篇新聞熱門斷詞關鍵字統計新聞摘要-整理並存檔

# All-in-one

一鍵完工

In [1]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('cna_category_news.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(w, p)) for w, p in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc', 'VC']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]

Tokenize OK!
CPU times: total: 51 s
Wall time: 15 s


In [2]:
## Read it out 讀出看看
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')
df.head(1)


,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,暫無,暫無,"[('媒體', 8), ('新聞', 8), ('蒙特內哥羅', 6), ('劉世忠', 5...","['駐', '匈牙利', '代表', '劉世忠', '日前', '接受', '蒙特內哥羅',...","['匈牙利', '代表', '劉世忠', '接受', '蒙特內哥羅', '媒體', '新聞'...","[NerToken(word='匈牙利', ner='GPE', idx=(1, 4)), ...","[('駐', 'P'), ('匈牙利', 'Nc'), ('代表', 'Na'), ('劉世...",/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...


# Demonstration Step by Step 

# Read data from file

In [3]:
import pandas as pd
import numpy

In [4]:
df = pd.read_csv('cna_category_news.csv', sep='|')

In [5]:
df.shape

(9, 7)

In [6]:
df.head()

,item_id,date,category,title,content,link,photo_link
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
1,aipl_20240312_2,2024-03-12,政治,拜登提2025年總預算 編列1億美元助台海安全,白宮今天公布2025財政年度預算案，重點包括增進美國在全球的嚇阻力，因應主要對手中國帶來的挑...,/news/aipl/202403120005.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
2,aipl_20240311_3,2024-03-11,政治,黃捷提修法 中配入籍須宣誓效忠台灣,針對中國大陸籍配偶入籍我國，民進黨立委黃捷提出「台灣地區與大陸地區人民關係條例」部分條文修法...,/news/aipl/202403110363.aspx,NaN
3,ait_20240309_1,2024-03-09,科技,警戒海域有船舶滯留 日本民間火箭被迫中止發射,日本太空新創公司SPACE ONE今天上午在小型火箭KAIROS一號機準備升空前臨時喊卡，宣...,/news/ait/202403090111.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
4,ait_20240311_2,2024-03-11,科技,馬斯克：旗下xAI聊天機器人Grok將開放原始碼,美國億萬富豪馬斯克今天表示，他的人工智慧（AI）新創公司xAI本週將釋出ChatGPT挑戰者...,/news/ait/202403110358.aspx,NaN


In [7]:
df.content[0:5]

0    駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...
1    白宮今天公布2025財政年度預算案，重點包括增進美國在全球的嚇阻力，因應主要對手中國帶來的挑...
2    針對中國大陸籍配偶入籍我國，民進黨立委黃捷提出「台灣地區與大陸地區人民關係條例」部分條文修法...
3    日本太空新創公司SPACE ONE今天上午在小型火箭KAIROS一號機準備升空前臨時喊卡，宣...
4    美國億萬富豪馬斯克今天表示，他的人工智慧（AI）新創公司xAI本週將釋出ChatGPT挑戰者...
Name: content, dtype: object

# Ckiplab word segment (中研院斷詞)

# Load tokenization models

    We provide three levels (1–3) of drivers. Level 1 is the fastest, and level 3 (default) is the most accurate.
    我們的工具分為三個等級（1—3）。等級一最快，等級三（預設值）最精準。




In [8]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [9]:
# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

## Word Segmentation

In [10]:
%%time
# It takes time.
tokens=ws(df.content)

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.74s/it]

CPU times: total: 10.6 s
Wall time: 1.77 s


In [11]:
len(tokens)

9

In [12]:
len(tokens[0])

473

In [13]:
tokens[0]

['駐',
 '匈牙利',
 '代表',
 '劉世忠',
 '日前',
 '接受',
 '蒙特內哥羅',
 '重要',
 '媒體',
 '「',
 '每日',
 '新聞',
 '」',
 '（',
 'Vijesti',
 '）',
 '專訪',
 '，',
 '報導',
 '刊出',
 '後',
 '遭',
 '中國',
 '駐',
 '蒙特內哥羅',
 '大使館',
 '抗議',
 '，',
 '要求',
 '撤除',
 '報導',
 '，',
 '對',
 '此',
 '每日',
 '新聞',
 '回應',
 '「',
 '無法',
 '移除',
 '」',
 '，',
 '堅守',
 '媒體',
 '獨立',
 '與',
 '自由',
 '。',
 '隨',
 '烏俄',
 '戰爭',
 '陷入',
 '僵局',
 '，',
 '東歐',
 '媒體',
 '關注',
 '當前',
 '國際',
 '衝突',
 '如何',
 '影響',
 '台海',
 '局勢',
 '，',
 '駐',
 '匈牙利',
 '代表',
 '劉世忠',
 '近日',
 '接受',
 '兼',
 '轄國',
 '多',
 '家',
 '媒體',
 '專訪',
 '，',
 '分析',
 '烏俄',
 '戰爭',
 '對',
 '台海局勢',
 '影響',
 '，',
 '並',
 '與',
 '巴爾幹',
 '地區',
 '讀者',
 '分享',
 '台灣',
 '民主',
 '和',
 '經貿',
 '發展',
 '經驗',
 '。',
 '針對',
 '台',
 '海',
 '局勢',
 '，',
 '劉世忠',
 '向',
 '蒙特內哥羅',
 '媒體',
 '「',
 '每日',
 '新聞',
 '」',
 '（',
 'Vijesti',
 '）',
 '說明',
 '，',
 '台灣',
 '已',
 '是',
 '主權',
 '獨立',
 '的',
 '民主',
 '國家',
 '，',
 '與',
 '中華',
 '人民',
 '共和',
 '國',
 '互',
 '不',
 '隸屬',
 '，',
 '亦',
 '無',
 '宣布',
 '獨立',
 '之',
 '必要',
 '，',
 '並',
 '強',
 '調',
 '俄烏',
 '戰爭',
 '的',
 

In [15]:
tokens[0][0]

'駐'

## POS

In [17]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]

CPU times: total: 20.1 s
Wall time: 3.37 s


In [18]:
len(tokens_pos)

9

In [19]:
len(tokens_pos[0])

473

In [20]:
tokens_pos[0]

['P',
 'Nc',
 'Na',
 'Nb',
 'Nd',
 'VC',
 'Nc',
 'VH',
 'Na',
 'PARENTHESISCATEGORY',
 'Nes',
 'Na',
 'PARENTHESISCATEGORY',
 'PARENTHESISCATEGORY',
 'FW',
 'PARENTHESISCATEGORY',
 'VC',
 'COMMACATEGORY',
 'VE',
 'VE',
 'Ng',
 'P',
 'Nc',
 'VCL',
 'Nc',
 'Nc',
 'VE',
 'COMMACATEGORY',
 'VF',
 'VC',
 'VE',
 'COMMACATEGORY',
 'P',
 'Nep',
 'Nes',
 'Na',
 'VC',
 'PARENTHESISCATEGORY',
 'D',
 'VC',
 'PARENTHESISCATEGORY',
 'COMMACATEGORY',
 'VJ',
 'Na',
 'VH',
 'Caa',
 'VH',
 'PERIODCATEGORY',
 'P',
 'Nc',
 'Na',
 'VCL',
 'Na',
 'COMMACATEGORY',
 'Nc',
 'Na',
 'VJ',
 'Nd',
 'Nc',
 'Na',
 'D',
 'VC',
 'Nc',
 'Na',
 'COMMACATEGORY',
 'VCL',
 'Nc',
 'Na',
 'Nb',
 'Nd',
 'VC',
 'VG',
 'Nc',
 'Neqa',
 'Nf',
 'Na',
 'VC',
 'COMMACATEGORY',
 'VC',
 'Nc',
 'Na',
 'P',
 'Nc',
 'VC',
 'COMMACATEGORY',
 'Cbb',
 'P',
 'Nc',
 'Nc',
 'Na',
 'VJ',
 'Nc',
 'Na',
 'Caa',
 'Na',
 'Na',
 'Na',
 'PERIODCATEGORY',
 'P',
 'Nc',
 'Nc',
 'Na',
 'COMMACATEGORY',
 'Nb',
 'P',
 'Nb',
 'Na',
 'PARENTHESISCATEGORY',
 

## word pos pair 詞性關鍵字

In [21]:
word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens, tokens_pos)]

In [22]:
len(word_pos_pair)

9

In [23]:
word_pos_pair[0][0]

('駐', 'P')

In [24]:
word_pos_pair

[[('駐', 'P'),
  ('匈牙利', 'Nc'),
  ('代表', 'Na'),
  ('劉世忠', 'Nb'),
  ('日前', 'Nd'),
  ('接受', 'VC'),
  ('蒙特內哥羅', 'Nc'),
  ('重要', 'VH'),
  ('媒體', 'Na'),
  ('「', 'PARENTHESISCATEGORY'),
  ('每日', 'Nes'),
  ('新聞', 'Na'),
  ('」', 'PARENTHESISCATEGORY'),
  ('（', 'PARENTHESISCATEGORY'),
  ('Vijesti', 'FW'),
  ('）', 'PARENTHESISCATEGORY'),
  ('專訪', 'VC'),
  ('，', 'COMMACATEGORY'),
  ('報導', 'VE'),
  ('刊出', 'VE'),
  ('後', 'Ng'),
  ('遭', 'P'),
  ('中國', 'Nc'),
  ('駐', 'VCL'),
  ('蒙特內哥羅', 'Nc'),
  ('大使館', 'Nc'),
  ('抗議', 'VE'),
  ('，', 'COMMACATEGORY'),
  ('要求', 'VF'),
  ('撤除', 'VC'),
  ('報導', 'VE'),
  ('，', 'COMMACATEGORY'),
  ('對', 'P'),
  ('此', 'Nep'),
  ('每日', 'Nes'),
  ('新聞', 'Na'),
  ('回應', 'VC'),
  ('「', 'PARENTHESISCATEGORY'),
  ('無法', 'D'),
  ('移除', 'VC'),
  ('」', 'PARENTHESISCATEGORY'),
  ('，', 'COMMACATEGORY'),
  ('堅守', 'VJ'),
  ('媒體', 'Na'),
  ('獨立', 'VH'),
  ('與', 'Caa'),
  ('自由', 'VH'),
  ('。', 'PERIODCATEGORY'),
  ('隨', 'P'),
  ('烏俄', 'Nc'),
  ('戰爭', 'Na'),
  ('陷入', 'VCL'),
  ('僵局', 'Na')

## NER命名實體辨識

In [25]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]

CPU times: total: 10.9 s
Wall time: 1.83 s


In [26]:
entity_list[0]

[NerToken(word='匈牙利', ner='GPE', idx=(1, 4)),
 NerToken(word='劉世忠', ner='PERSON', idx=(6, 9)),
 NerToken(word='蒙特內哥羅', ner='GPE', idx=(13, 18)),
 NerToken(word='中國', ner='GPE', idx=(46, 48)),
 NerToken(word='蒙特內哥羅', ner='GPE', idx=(49, 54)),
 NerToken(word='烏俄戰爭', ner='EVENT', idx=(93, 97)),
 NerToken(word='台海', ner='GPE', idx=(118, 120)),
 NerToken(word='劉世忠', ner='PERSON', idx=(129, 132)),
 NerToken(word='巴爾幹', ner='LOC', idx=(162, 165)),
 NerToken(word='台灣', ner='GPE', idx=(171, 173)),
 NerToken(word='台', ner='GPE', idx=(185, 186)),
 NerToken(word='海', ner='GPE', idx=(186, 187)),
 NerToken(word='劉世忠', ner='PERSON', idx=(190, 193)),
 NerToken(word='蒙特內哥羅', ner='PERSON', idx=(194, 199)),
 NerToken(word='Vijest', ner='PERSON', idx=(208, 214)),
 NerToken(word='中華人民共和國', ner='GPE', idx=(234, 241)),
 NerToken(word='俄', ner='GPE', idx=(259, 260)),
 NerToken(word='台灣人', ner='NORP', idx=(267, 270)),
 NerToken(word='劉世忠', ner='PERSON', idx=(296, 299)),
 NerToken(word='台', ner='GPE', idx=(302,

# Remove stop words and filter using POS (tokens_v2)

去除停用詞並依據詞性過濾

In [45]:
# with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#     stops = f.read().split('\n') 

In [27]:
# 過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

In [28]:
tokens_v2[0]

['匈牙利',
 '代表',
 '劉世忠',
 '蒙特內哥羅',
 '媒體',
 '新聞',
 '中國',
 '蒙特內哥羅',
 '大使館',
 '新聞',
 '媒體',
 '烏俄',
 '戰爭',
 '僵局',
 '東歐',
 '媒體',
 '國際',
 '衝突',
 '台海',
 '局勢',
 '匈牙利',
 '代表',
 '劉世忠',
 '轄國',
 '媒體',
 '烏俄',
 '戰爭',
 '台海局勢',
 '巴爾幹',
 '地區',
 '讀者',
 '台灣',
 '民主',
 '經貿',
 '發展',
 '經驗',
 '局勢',
 '劉世忠',
 '蒙特內哥羅',
 '媒體',
 '每日',
 '新聞',
 '台灣',
 '主權',
 '民主',
 '國家',
 '中華',
 '人民',
 '共和',
 '必要',
 '俄烏',
 '戰爭',
 '台灣人',
 '威權',
 '國家',
 '民主生活',
 '方式',
 '決心',
 '劉世忠',
 '台海',
 '局勢',
 '利益',
 '歐洲',
 '國家',
 '國際',
 '社會',
 '成員',
 '中國',
 '軍事',
 '手段',
 '台海',
 '現狀',
 '每日新聞',
 '北京',
 '巴爾幹',
 '半島',
 '響力',
 '問題',
 '劉世忠',
 '中國',
 '巴爾幹',
 '地區',
 '國家',
 '債務',
 '危機',
 '成效',
 '台灣',
 '國際',
 '社會',
 '力量',
 '方式',
 '巴爾幹國家',
 '台灣',
 '蒙特內哥羅',
 '巴爾幹島',
 '國家',
 '歐洲',
 '銀行',
 '職業',
 '訓練',
 '計畫',
 '當地',
 '學子',
 '巴爾幹',
 '地區',
 '產官學',
 '人士',
 '世界',
 '智慧',
 '城市',
 '論壇',
 '台灣',
 '經驗',
 '紙本',
 '中國',
 '蒙特內哥羅',
 '大使館',
 '要求',
 '每日',
 '新聞',
 '新聞',
 '蒙語',
 '英文',
 '網站',
 '人次',
 '每日',
 '新聞',
 '蒙特內哥羅',
 '媒體',
 '集團',
 '歐派',
 '媒體',
 '記者',
 '拉基奇',
 '長期',
 '當地',
 '政

# Insert tokens into dataframe (新增斷詞資料欄位)

In [29]:
df['tokens'] = tokens

In [30]:
df['tokens_v2'] = tokens_v2

In [31]:
df['entities'] = entity_list

In [32]:
df['token_pos'] = word_pos_pair

In [33]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[駐, 匈牙利, 代表, 劉世忠, 日前, 接受, 蒙特內哥羅, 重要, 媒體, 「, 每日...","[匈牙利, 代表, 劉世忠, 蒙特內哥羅, 媒體, 新聞, 中國, 蒙特內哥羅, 大使館, ...","[(匈牙利, GPE, (1, 4)), (劉世忠, PERSON, (6, 9)), (蒙...","[(駐, P), (匈牙利, Nc), (代表, Na), (劉世忠, Nb), (日前, ..."


# Calculate word count (frequency) 計算字頻(次數)

In [36]:
# allowPOS過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']

from collections import Counter
def word_frequency( wp_pair ):
    filtered_words =[]
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter( filtered_words)
    return counter.most_common(20)

In [37]:
word_pos_pair[0]

[('駐', 'P'),
 ('匈牙利', 'Nc'),
 ('代表', 'Na'),
 ('劉世忠', 'Nb'),
 ('日前', 'Nd'),
 ('接受', 'VC'),
 ('蒙特內哥羅', 'Nc'),
 ('重要', 'VH'),
 ('媒體', 'Na'),
 ('「', 'PARENTHESISCATEGORY'),
 ('每日', 'Nes'),
 ('新聞', 'Na'),
 ('」', 'PARENTHESISCATEGORY'),
 ('（', 'PARENTHESISCATEGORY'),
 ('Vijesti', 'FW'),
 ('）', 'PARENTHESISCATEGORY'),
 ('專訪', 'VC'),
 ('，', 'COMMACATEGORY'),
 ('報導', 'VE'),
 ('刊出', 'VE'),
 ('後', 'Ng'),
 ('遭', 'P'),
 ('中國', 'Nc'),
 ('駐', 'VCL'),
 ('蒙特內哥羅', 'Nc'),
 ('大使館', 'Nc'),
 ('抗議', 'VE'),
 ('，', 'COMMACATEGORY'),
 ('要求', 'VF'),
 ('撤除', 'VC'),
 ('報導', 'VE'),
 ('，', 'COMMACATEGORY'),
 ('對', 'P'),
 ('此', 'Nep'),
 ('每日', 'Nes'),
 ('新聞', 'Na'),
 ('回應', 'VC'),
 ('「', 'PARENTHESISCATEGORY'),
 ('無法', 'D'),
 ('移除', 'VC'),
 ('」', 'PARENTHESISCATEGORY'),
 ('，', 'COMMACATEGORY'),
 ('堅守', 'VJ'),
 ('媒體', 'Na'),
 ('獨立', 'VH'),
 ('與', 'Caa'),
 ('自由', 'VH'),
 ('。', 'PERIODCATEGORY'),
 ('隨', 'P'),
 ('烏俄', 'Nc'),
 ('戰爭', 'Na'),
 ('陷入', 'VCL'),
 ('僵局', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('東歐', 'Nc'),
 ('媒體', 'Na

In [39]:
word_frequency(word_pos_pair[3])

[('發射', 9),
 ('火箭', 5),
 ('日本', 3),
 ('升空', 3),
 ('船舶', 3),
 ('小型', 2),
 ('延期', 2),
 ('滯留', 2),
 ('民間', 2),
 ('發射場', 2),
 ('紀伊', 2),
 ('衛星', 2),
 ('人員', 2),
 ('太空', 1),
 ('新創', 1),
 ('公司', 1),
 ('原因', 1),
 ('海域', 1),
 ('和歌山縣', 1),
 ('串本町', 1)]

In [40]:
## 逐篇計算詞頻率

In [41]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

CPU times: total: 0 ns
Wall time: 1 ms


In [42]:
keyfreqs[0:1]

[[('媒體', 8),
  ('新聞', 8),
  ('蒙特內哥羅', 6),
  ('劉世忠', 5),
  ('台灣', 5),
  ('國家', 5),
  ('專訪', 4),
  ('中國', 4),
  ('國際', 4),
  ('巴爾幹', 4),
  ('戰爭', 3),
  ('台海', 3),
  ('局勢', 3),
  ('地區', 3),
  ('每日', 3),
  ('社會', 3),
  ('匈牙利', 2),
  ('代表', 2),
  ('接受', 2),
  ('大使館', 2)]]

In [43]:
df['top_key_freq'] = keyfreqs

In [44]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[駐, 匈牙利, 代表, 劉世忠, 日前, 接受, 蒙特內哥羅, 重要, 媒體, 「, 每日...","[匈牙利, 代表, 劉世忠, 蒙特內哥羅, 媒體, 新聞, 中國, 蒙特內哥羅, 大使館, ...","[(匈牙利, GPE, (1, 4)), (劉世忠, PERSON, (6, 9)), (蒙...","[(駐, P), (匈牙利, Nc), (代表, Na), (劉世忠, Nb), (日前, ...","[(媒體, 8), (新聞, 8), (蒙特內哥羅, 6), (劉世忠, 5), (台灣, ..."


In [45]:
df.iloc[0].top_key_freq

[('媒體', 8),
 ('新聞', 8),
 ('蒙特內哥羅', 6),
 ('劉世忠', 5),
 ('台灣', 5),
 ('國家', 5),
 ('專訪', 4),
 ('中國', 4),
 ('國際', 4),
 ('巴爾幹', 4),
 ('戰爭', 3),
 ('台海', 3),
 ('局勢', 3),
 ('地區', 3),
 ('每日', 3),
 ('社會', 3),
 ('匈牙利', 2),
 ('代表', 2),
 ('接受', 2),
 ('大使館', 2)]

# Abstract (summary) and sentimental score(摘要與情緒分數)

Currently we don't use abstract in our later application. However, we prepare news summary for future usage.

We use snowNLP to get summary and sentimental score.

In [61]:
# pip install snownlp

In [47]:
%%time
summary=[]
sentiment=[]
for text in df.content: # process piece by piece
    summary.append("暫無")  
    sentiment.append("暫無")

CPU times: total: 0 ns
Wall time: 973 µs


In [48]:
df['summary'] = summary

In [49]:
df['sentiment'] = sentiment

In [50]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq,summary,sentiment
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"[駐, 匈牙利, 代表, 劉世忠, 日前, 接受, 蒙特內哥羅, 重要, 媒體, 「, 每日...","[匈牙利, 代表, 劉世忠, 蒙特內哥羅, 媒體, 新聞, 中國, 蒙特內哥羅, 大使館, ...","[(匈牙利, GPE, (1, 4)), (劉世忠, PERSON, (6, 9)), (蒙...","[(駐, P), (匈牙利, Nc), (代表, Na), (劉世忠, Nb), (日前, ...","[(媒體, 8), (新聞, 8), (蒙特內哥羅, 6), (劉世忠, 5), (台灣, ...",暫無,暫無


# Rearrange the colmun order for readability

In [51]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq',
       'summary', 'sentiment'],
      dtype='object')

In [52]:
# This operation is the same as slicing
df=df[['item_id', 'date', 'category', 'title', 'content','sentiment', 'summary', 'top_key_freq', 'tokens',
       'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link'
       ]]

In [53]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,暫無,暫無,"[(媒體, 8), (新聞, 8), (蒙特內哥羅, 6), (劉世忠, 5), (台灣, ...","[駐, 匈牙利, 代表, 劉世忠, 日前, 接受, 蒙特內哥羅, 重要, 媒體, 「, 每日...","[匈牙利, 代表, 劉世忠, 蒙特內哥羅, 媒體, 新聞, 中國, 蒙特內哥羅, 大使館, ...","[(匈牙利, GPE, (1, 4)), (劉世忠, PERSON, (6, 9)), (蒙...","[(駐, P), (匈牙利, Nc), (代表, Na), (劉世忠, Nb), (日前, ...",/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...


# Save data to disk

存檔

In [54]:
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看

In [55]:
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')

In [56]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20240312_1,2024-03-12,政治,蒙特內哥羅媒體不畏中國施壓 拒絕撤除劉世忠專訪,駐匈牙利代表劉世忠日前接受蒙特內哥羅重要媒體「每日新聞」（Vijesti）專訪，報導刊出後遭...,暫無,暫無,"[('媒體', 8), ('新聞', 8), ('蒙特內哥羅', 6), ('劉世忠', 5...","['駐', '匈牙利', '代表', '劉世忠', '日前', '接受', '蒙特內哥羅',...","['匈牙利', '代表', '劉世忠', '蒙特內哥羅', '媒體', '新聞', '中國'...","[NerToken(word='匈牙利', ner='GPE', idx=(1, 4)), ...","[('駐', 'P'), ('匈牙利', 'Nc'), ('代表', 'Na'), ('劉世...",/news/aipl/202403120014.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
